# Deepface Face Verification test 

- Import relevant libraries:

In [105]:
# !pip install deepface
from deepface import DeepFace
from deepface_test_functions import find_image, deepface_benchmark_lfw_split, deepface_benchmark, deepface_get_accuracy, deepface_run_dataset

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

import os

# Benchmark Datasets
benchmark_df = pd.read_csv('./data/LFW-csv/pairs.csv')
benchmark_balance = pd.read_csv('./data/LFW-csv/pairs_balance.csv')
# benchmark_balance_mod = pd.read_csv('./data/LFW-csv/pairs_balance_mod.csv')
benchmark_deepface_df = pd.read_csv('./data/LFW-csv/pairs_benchmark_deepface.csv')


# Results:
result_df = pd.read_csv('./result/result_deepface.csv')
result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')
# result_benchmark_mod = pd.read_csv('./result/deepface_benchmark_mod.csv')

result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

# Deepface Threshold results:

In [4]:
## Results based on https://youtu.be/i_MOwvhbLdI 
_vgg_face = {"Model": "VGGFace", "Accuracy": 89.28, "Precision": 97.41, "Recall": 80.71}
_facenet = {"Model": "FaceNet", "Accuracy": 98.21, "Precision": 100, "Recall": 96.42}
_openface = {"Model": "OpenFace", "Accuracy": 57.85, "Precision": 95.83, "Recall": 16.42}
_deepface = {"Model": "DeepFace", "Accuracy": 54.64, "Precision": 100, "Recall": 9.28}

models_results = [_facenet, _openface, _deepface]

result_model_df = pd.DataFrame([_vgg_face])
for model in models_results:
    result_model = pd.DataFrame([model])
    frames = [result_model_df, result_model]
    result_model_df = pd.concat(frames)

    result_model_df = result_model_df.reset_index()
    result_model_df.drop(columns=['index'], inplace=True)

result_model_df

,Model,Accuracy,Precision,Recall
0,VGGFace,89.28,97.41,80.71
1,FaceNet,98.21,100.00,96.42
2,OpenFace,57.85,95.83,16.42
3,DeepFace,54.64,100.00,9.28


## Paramater settings:

In [98]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# test with gender split LFW
# LFW_gender, LFW_gender_makeup_heavy, LFW_gender_makeup_light
DATASET_LFW_SPLIT = "LFW_gender_makeup_light"

# select a model to run:
# models = ["Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "Facenet512"

# Faces are represented as vectors, a face pair of same person should be similar, calculated by different metrics
# metrics = ["cosine", "euclidean", "euclidean_l2"]
METRIC = "euclidean_l2"

# Face detector, backend that helps aligning the face for better accuracy
# backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
BACKEND = 'opencv'

# Select 'Male' or 'Female'
TEST_GENDER = 'Female'

# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/{DATASET_LFW_SPLIT}"

## Deepface test: 

- Manual test:

In [ ]:
# benchmark testing
# benchmark_df          # 6000 images
# benchmark_deepface_df # 2000 images
deepface_benchmark_res = deepface_benchmark(DATASET, dir_path, benchmark_deepface_df, MODEL, METRIC, BACKEND)

- Benchmark testing with LFW gender split

In [99]:
# benchmark testing LFW with gender split 
# benchmark_balance         # 3000 images
# benchmark_balance_mod     # 3000 images

# currrently using:  
# benchmark_deepface_df     # 2000 images

deepface_benchmark_res, perturbed_data = deepface_benchmark_lfw_split(DATASET_LFW_SPLIT, dir_path, benchmark_deepface_df,\
                                                                     MODEL, METRIC, BACKEND, TEST_GENDER)

1/1 [==============================] - 0s 51ms/step


In [14]:
# import shutil

# # gender = "Male"
# gender = "Female"

# dst = f'../makeup_application/sample_images/{gender}/test'
# for data in perturbed_data:
#     if data["img_path_1"] == None:
#         match = re.search(f"_\d.*\.jpg", data["img1"])
#         NAME = data["img1"].replace(match.group(0), "")
#         src = f'./data/LFW_gender/{gender}/{NAME}/{data["img1"]}'
#         shutil.copy2(src, dst)
#     elif data["img_path_2"] == None:
#         match = re.search(f"_\d.*\.jpg", data["img2"])
#         NAME = data["img2"].replace(match.group(0), "")
#         src = f'./data/LFW_gender/{gender}/{NAME}/{data["img2"]}'
#         shutil.copy2(src, dst)

## Result of the Deepface test:

- Manual testing:

- Benchmark testing:

In [100]:
# confusion matrix accuracy:
cm_acc = deepface_benchmark_res["CM_ACC"]
cm_pre = deepface_benchmark_res["Precision"]
cm_rec = deepface_benchmark_res["Recall"]
truth_pos = deepface_benchmark_res["TP"]
truth_neg = deepface_benchmark_res["TN"]
false_pos = deepface_benchmark_res["FP"]
false_neg = deepface_benchmark_res["FN"]
undetected = deepface_benchmark_res["Undetected"]

# total images
total_imgs = deepface_benchmark_res["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================\n\
Undetected: {undetected}")

Accuracy of Facenet512 is 90.0% out of 1000 images from LFW dataset.
The Precision is 100.0 and Recall is 81.0
Truth Positive: 405 
Truth Negative: 498 
False Positive: 2 
False Negative: 95 
Undetected: 0


# Saving result:

In [101]:
result_df_new = pd.DataFrame([deepface_benchmark_res])

# result_df
# result_benchmark_gender
# result_benchmark_gender_perturbed
# result_benchmark_gender_nonperturbed

frames = [result_benchmark_gender_perturbed, result_df_new]
result_benchmark_df = pd.concat(frames)

result_benchmark_df = result_benchmark_df.reset_index()
result_benchmark_df.drop(columns=['index'], inplace=True)
result_benchmark_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0


- Write to CSV for Benchmark original LFW dataset 

In [ ]:
result_benchmark_df.to_csv("./result/result_deepface.csv", index=False)
result_df = pd.read_csv('./result/result_deepface.csv')

- Write to CSV for Benchmark LFW gender split

In [ ]:
# Write results to csv file
# result_benchmark_df.to_csv("./result/deepface_benchmark_gender.csv", index=False)
# result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')

result_benchmark_df.to_csv("./result/deepface_benchmark_gender_nonperturbed.csv", index=False)
result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')

In [ ]:
result_benchmark_gender_nonperturbed

- Write to CSV for perturbed data Benchmark LFW gender split

In [102]:
# Write results to csv file
result_benchmark_df.to_csv("./result/deepface_benchmark_gender_perturbed.csv", index=False)
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

In [106]:
result_benchmark_gender_perturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0


### Etc testing